In [21]:
import os
import shutil
import pprint
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

In [22]:
calc_csv = "calc_case(with_jpg_img).csv"
mass_csv = "mass_case(with_jpg_img).csv"

In [23]:
sub_direc = ["Test", "Train"]
output_dir = os.path.normpath("D:\Projects\DL\Breast-Cancer-Detection\dataset")

In [24]:
path = "D:\Projects\DL\Breast-Cancer-Detection\dataset"
norm_path = os.path.normpath(path)
os.chdir(norm_path)

In [25]:
os.mkdir('Train')
os.mkdir('Test')

In [26]:
csv_calc = pd.read_csv(calc_csv)
csv_mass = pd.read_csv(mass_csv)

In [27]:
print(os.getcwd())

D:\Projects\DL\Breast-Cancer-Detection\dataset


In [29]:
# The function cannot tell that it has reached to the end of the folder and directly jumps to the exception

# demo path = "/kaggle/input/breast-cancer-jpg-image-dataset-of-cbisddsm/k_CBIS-DDSM/jpg_img/Calc_Test_P_00038_LEFT_CC-1.3.6.1.4.1.9590.100.1.2.85935434310203356712688695661986996009-1.3.6.1.4.1.9590.100.1.2.374115997511889073021386151921807063992"
def move_jpg(source_dir):
    count = 0
    for direc in os.listdir(source_dir):
        if "Calc_Test" in direc:
            dest_dir = "\Projects\DL\Breast-Cancer-Detection\dataset\Test"
        elif "Calc_Train" in direc:
            dest_dir = '\Projects\DL\Breast-Cancer-Detection\dataset\Train'
        elif "Mass_Test" in direc:
            dest_dir = '\Projects\DL\Breast-Cancer-Detection\dataset\Test'
        else:
            dest_dir = '\Projects\DL\Breast-Cancer-Detection\dataset\Train'
        src = os.path.normpath(os.path.join(source_dir, direc))
        dest = os.path.normpath(os.path.join(dest_dir, direc))
        shutil.copytree(src,dest)
        dest_dir = ""
        count += 1
    print("Folder Count: ", count)
            

In [30]:
source_dir = "jpg_img/"

In [31]:
move_jpg(source_dir)

Folder Count:  6775


In [32]:
# doesnot rename the first file ????
def new_name_jpg(source_dir, csv_calc, csv_mass):
    class_name = ""
    try:
        for direc in os.listdir(source_dir):
            try:
                patientID = direc[direc.find("P_"): direc.find("P_")+7]
            except Exception as e:
                print(f"Cannot read patientID- {str(e)}")
            
            try:
                if "Calc" in direc:
                    tumor_type = "calcification"
                    df = csv_calc
                elif "Mass" in direc:
                    tumor_type = "mass"
                    df = csv_mass
                else:
                    print("Error: No tumor information.")
            except Exception as e:
                print(f"Cannot determine tumor type- {str(e)}")
                
            try:
                if "Test" in direc:
                    data_split = "test"
                elif "Train" in direc:
                    data_split = "train"
                else:
                    print("Error: No data split information.")
            except Exception as e:
                print(f"Cannot determine data split-{str(e)}")
            try:
                if "LEFT" in direc:
                    side = "LEFT"
                elif "RIGHT" in direc:
                    side = "RIGHT"
                else:
                    print("Error: No side information.")
            
                if "CC" in direc:
                    view = "CC"
                elif "MLO" in direc:
                    view= "MLO"
                else:
                    print("Error: No view information.")
            except Exception as e:
                print(f"Failure in getting side and view-{str(e)}")
                            
            # Getting the class_name
            index = df.loc[(df['patient_id'] == patientID) & (df['left or right breast'] == side) & 
                           (df['image view'] == view) & (df['abnormality type'] == tumor_type)].index

            # Check if there's a matching row in the DataFrame
            if not index.empty:
                class_name = str(df['pathology'].iloc[index[0]])  # Access and convert to string
            else:
                print("No matching row found in the DataFrame for class_name.")
        
            try:
                for files in os.listdir(os.path.normpath(os.path.join(source_dir + '/' + direc))):
                    counter=1
                    try:
                        if "full" in files:
                            img_type = "FULL"
                        elif "crop" in files:
                            img_type = "CROP"
                        elif "mask" in files:
                            img_type = "MASK"
                    except Exception as e:
                        print(f"Error in reading img_type-{str(e)}")
                    try:
                        new_name = tumor_type + '_' + data_split +'_'+ patientID + '_' + side + '_' + view + '_' + img_type + '_' + class_name +'_'+str(counter)+'.jpg'
                        counter += 1
                        src_dir = os.path.normpath(os.path.join(source_dir, direc))
                        # dest_dir = os.path.normpath(os.path.join(source_dir, direc, new_name))
                        try:
                            os.chdir(src_dir)
                            os.rename(files, new_name)
                            os.chdir('..')
                        except Exception as e:
                            print(f"Error in the renaming-{str(e)}")
                    except Exception as e:
                        print(f"Error in assigning name-{str(e)}")
            except Exception as e:
                print(f"Error in getting files inside the sub-directory-{str(e)}")
    except Exception as e:
        print(f"Cannot read each directories-{str(e)}")

In [33]:
def organize(source_dir):
    for direc in os.listdir(source_dir):
        full_direc_path = os.path.join(source_dir, direc)  # Full path to subdirectory
        if os.path.isdir(full_direc_path):
            for files in os.listdir(full_direc_path):
                source = os.path.join(full_direc_path, files)
                filename, file_extension = os.path.splitext(files)
                timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
                new_filename = f"{filename}_{timestamp}{file_extension}"
                dest = os.path.join(source_dir, new_filename)
                shutil.move(source, dest)
            os.rmdir(full_direc_path)

In [34]:
for sub_dir in sub_direc:
    organize(os.path.join(output_dir, sub_dir))

In [35]:
os.chdir(os.path.normpath(os.path.join(output_dir, "Test")))
os.mkdir("Malignant")
os.mkdir("Benign")
os.chdir("..")
os.chdir("Train")
os.mkdir("Malignant")
os.mkdir("Benign")
os.chdir("..")

In [36]:
for sub_dir in sub_direc:
    full_path = os.path.join(output_dir, sub_dir)

    for file in os.listdir(full_path):
        if "MALIGNANT" in file:
            src = os.path.join(full_path, file)
            dest = os.path.join(full_path, "Malignant")
            shutil.move(src, dest)
        elif "BENIGN" in file:
            src = os.path.join(full_path, file)
            dest = os.path.join(full_path, "Benign")
            shutil.move(src, dest)
        else:
            print(f"Error in getting class_name from file: {file}")



Error in getting class_name from file: Benign
Error in getting class_name from file: cropped-images-img_0-1_20231104221513.jpg
Error in getting class_name from file: cropped-images-img_0-1_20231104221514.jpg
Error in getting class_name from file: cropped-images-img_0-1_20231104221515.jpg
Error in getting class_name from file: cropped-images-img_0-1_20231104221516.jpg
Error in getting class_name from file: cropped-images-img_0-1_20231104221518.jpg
Error in getting class_name from file: cropped-images-img_0-2_20231104221512.jpg
Error in getting class_name from file: cropped-images-img_1-2_20231104221512.jpg
Error in getting class_name from file: full-mammogram-images-img_0-1_20231104221507.jpg
Error in getting class_name from file: full-mammogram-images-img_0-1_20231104221508.jpg
Error in getting class_name from file: full-mammogram-images-img_0-1_20231104221509.jpg
Error in getting class_name from file: full-mammogram-images-img_0-1_20231104221510.jpg
Error in getting class_name from fi